In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///croot/scikit-learn_1684954695550/work


In [3]:
import sklearn
sklearn.__version__

'1.2.2'

In [2]:
!python -V

Python 3.11.7


In [5]:
import pickle
import pandas as pd

In [7]:
pd.__version__

'2.1.4'

In [9]:
import pyarrow
pyarrow.__version__

'14.0.2'

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/markgolovko/anaconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/markgolovko/anaconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [11]:
f'standard deviation of the predicted duration is {round(y_pred.std(),2)}'

'standard deviation of the predicted duration is 6.25'

In [16]:
df['ride_id'] = f'2023/03_' + df.index.astype('str')

In [20]:
df_result = pd.DataFrame({
    'ride_id': df['ride_id'],
    'predictions': y_pred
})

output_file = 'output.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)